In [28]:
from tensorflow.python.client import device_lib
from keras.models import Sequential
from keras.layers import Dropout, Dense
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
stop = set(stopwords.words('english'))
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import json
import ast
import eli5
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
#from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import time
import gc
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
#import shap
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9565827378234848944
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9117697966
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13687456816177258107
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [47]:
#train = pd.read_csv('dataset-0510/train.csv')
#test  = pd.read_csv('dataset-0510/test.csv')

train = pd.read_csv('FE_train.csv')
test  = pd.read_csv('FE_test.csv')
data  = pd.concat([train, test], axis=0)


#drop outlier
train.drop(train[(train['land_area'] > 1500) | (train['building_area'] >1000)].index, inplace= True)
'''
target_df = train.groupby(['city', 'town', 'building_type']).agg({'building_area' : ['mean', 'median'], 'land_area' : ['mean', 'median'], 'total_price' : ['mean', 'median']}).reset_index()
target_df.columns = [i[0] + '_' + i[1]  if i[1] != '' else i[0] for i in target_df.columns.tolist()]

target_df['total_price_median'] = np.log1p(target_df['total_price_median'])
target_df['price_land_rate_median'] = target_df['total_price_median'] / target_df['land_area_median']
target_df['price_building_rate_median'] = target_df['total_price_median'] / target_df['building_area_median']


combine_cols = ['city', 'town', 'building_type', 'price_land_rate_median', 'price_building_rate_median']
train = pd.merge(train, target_df[combine_cols], on =['city', 'town', 'building_type'], how='left')
test = pd.merge(test, target_df[combine_cols], on =['city', 'town', 'building_type'], how='left')


target_df = train.groupby('building_type')['total_price'].median().reset_index()
target_df['building_total_price'] = np.log1p(target_df['total_price'])
target_df.drop('total_price', axis =1, inplace = True)
train = pd.merge(train, target_df, on =['building_type'], how='left')
test = pd.merge(test, target_df, on =['building_type'], how='left')

train.loc[train['building_area'] == 4, 'parking_area'] = train.loc[train['building_area'] == 4, 'building_area'] / train.loc[train['building_area'] == 4, 'total_floor']
test.loc[train['building_area'] == 4, 'parking_area'] = test.loc[test['building_area'] == 4, 'building_area'] / test.loc[test['building_area'] == 4, 'total_floor']
'''
#drop_cols = [i for i in train.columns if np.sum(train[i]) == 60000 and 'index' in i]
train.drop([ 'inter_btw_building_type_building_use',
 'inter_btw_building_type_building_material',
 'inter_btw_building_type_parking_way',
 'inter_btw_building_use_building_material',
 'inter_btw_building_use_parking_way',
 'inter_btw_building_material_parking_way', 'building_type', 'parking_way','building_material', 'building_use'], axis = 1, inplace = True)
test.drop([ 'inter_btw_building_type_building_use',
 'inter_btw_building_type_building_material',
 'inter_btw_building_type_parking_way',
 'inter_btw_building_use_building_material',
 'inter_btw_building_use_parking_way',
 'inter_btw_building_material_parking_way', 'building_type', 'parking_way','building_material', 'building_use'], axis = 1, inplace = True)
#train.drop(drop_cols, axis = 1, inplace = True)
#test.drop(drop_cols, axis = 1, inplace = True)
gc.collect()

29

In [52]:
features = [i for i in train.columns if i not in ['building_id', 'total_price','total_price_log', 'city', 'location_2', 'parking_area', 'village_income_median', 'rate_500_50']]

In [66]:
StandardScaler().fit(train[features])

ValueError: Input contains infinity or a value too large for dtype('float64').

In [41]:
train['III_50'][:5] / train['III_10'][:5]

0    inf
1    1.0
2    NaN
3    inf
4    NaN
dtype: float64

In [39]:
0 / 0

ZeroDivisionError: division by zero

In [44]:
[i for i in train.columns if 'rate' in i]

['bachelor_rate',
 'born_rate',
 'death_rate',
 'divorce_rate',
 'doc_rate',
 'elementary_rate',
 'highschool_rate',
 'jobschool_rate',
 'junior_rate',
 'marriage_rate',
 'master_rate',
 'rate_between_txn_complete',
 'rate_500_50',
 'rate_1000_500',
 'rate_5000_1000',
 'rate_10000_5000']

In [38]:
train[['III_50_III_10_rate', 'III_50', 'III_10']]

,III_50_III_10_rate,III_50,III_10
0,inf,1,0
1,1.0,1,1
2,NaN,0,0
3,inf,1,0
4,NaN,0,0
5,NaN,0,0
6,NaN,0,0
7,inf,1,0
8,inf,1,0
9,inf,1,0


In [46]:
train['III_50_III_10_diff']

0        inf
1        1.0
2        NaN
3        inf
4        NaN
5        NaN
6        NaN
7        inf
8        inf
9        inf
10       NaN
11       NaN
12       inf
13       NaN
14       NaN
15       inf
16       NaN
17       NaN
18       inf
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       inf
26       NaN
27       NaN
28       NaN
29       NaN
        ... 
59970    inf
59971    inf
59972    NaN
59973    NaN
59974    NaN
59975    NaN
59976    NaN
59977    inf
59978    NaN
59979    NaN
59980    inf
59981    NaN
59982    NaN
59983    inf
59984    NaN
59985    NaN
59986    NaN
59987    inf
59988    NaN
59989    NaN
59990    inf
59991    NaN
59992    NaN
59993    NaN
59994    NaN
59995    NaN
59996    inf
59997    NaN
59998    NaN
59999    NaN
Name: III_50_III_10_diff, Length: 59989, dtype: float64

In [48]:
train.columns[train.isna().sum() > 0]

Index(['parking_area', 'village_income_median', 'rate_500_50'], dtype='object')

In [4]:
train['total_price_log'] = np.log1p(train['total_price'])
features = [i for i in train.columns if i not in ['building_id', 'total_price','total_price_log', 'city', 'location_2']] 
X = train[features].values
Y = train['total_price_log'].values

model = Sequential()
model.add(Dense(128, kernel_initializer='uniform',input_dim = train[features].shape[1], activation='sigmoid'))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='uniform',activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(500, kernel_initializer='uniform',activation='sigmoid'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss=rmse, optimizer='adam', metrics=[rmse])
model.fit(X, Y, epochs = 500, batch_size = 5000, validation_split = 0.2, verbose=2)


Train on 47991 samples, validate on 11998 samples
Epoch 1/500
 - 1s - loss: 12.0486 - rmse: 12.0486 - val_loss: 7.8895 - val_rmse: 7.8895
Epoch 2/500
 - 0s - loss: 4.7192 - rmse: 4.7192 - val_loss: 1.1086 - val_rmse: 1.1086
Epoch 3/500
 - 0s - loss: 1.3348 - rmse: 1.3348 - val_loss: 1.7585 - val_rmse: 1.7585
Epoch 4/500
 - 0s - loss: 1.3324 - rmse: 1.3324 - val_loss: 0.9318 - val_rmse: 0.9318
Epoch 5/500
 - 0s - loss: 1.0038 - rmse: 1.0038 - val_loss: 0.9865 - val_rmse: 0.9865
Epoch 6/500
 - 0s - loss: 0.9636 - rmse: 0.9636 - val_loss: 0.9451 - val_rmse: 0.9451
Epoch 7/500
 - 0s - loss: 0.9490 - rmse: 0.9490 - val_loss: 0.9331 - val_rmse: 0.9331
Epoch 8/500
 - 0s - loss: 0.9338 - rmse: 0.9338 - val_loss: 0.9194 - val_rmse: 0.9194
Epoch 9/500
 - 0s - loss: 0.9314 - rmse: 0.9314 - val_loss: 0.9255 - val_rmse: 0.9255
Epoch 10/500
 - 0s - loss: 0.9289 - rmse: 0.9289 - val_loss: 0.9218 - val_rmse: 0.9218
Epoch 11/500
 - 0s - loss: 0.9284 - rmse: 0.9284 - val_loss: 0.9210 - val_rmse: 0.9210


KeyboardInterrupt: 

In [28]:
temp = model.predict(X)

In [61]:
train[[i for i in features if train[i].dtypes == 'float64']] = round(train[[i for i in features if train[i].dtypes == 'float64']],3)

In [65]:
train[[i for i in features if train[i].dtypes == 'float64']] = train[[i for i in features if train[i].dtypes == 'float64']].astype('float32')